In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

# --- Simulation PEA ---
def simulate_pea(initial, monthly, rate_mean, rate_vol, months, dividend_yield=0.02, reinvest_dividends=True, tax_rate=0.172):
    values = []
    cash_dividends = []  # pour les dividendes encaissés
    total_invested = initial
    value = initial
    rng = np.random.default_rng()
    
    for i in range(months):
        # Rendement mensuel aléatoire
        rate = rng.normal(rate_mean, rate_vol)
        value = value * (1 + rate) + monthly
        total_invested += monthly
        
        # Dividendes annuels
        if (i + 1) % 12 == 0 and dividend_yield > 0:
            dividends = value * dividend_yield
            if reinvest_dividends:
                value += dividends
                cash_dividends.append(0)
            else:
                cash_dividends.append(dividends)
        else:
            cash_dividends.append(0)
        
        values.append(value)
    
    gain = value - total_invested
    tax = gain * tax_rate if gain > 0 else 0
    net_value = value - tax
    net_gain = gain - tax
    
    # Courbe nette mois par mois (prorata fiscalité)
    values_net = [v - ((v - total_invested) * tax_rate) for v in values]
    
    return net_value, total_invested, net_gain, tax, values, values_net, cash_dividends

# --- Simulation CTO ---
def simulate_cto(initial, monthly, rate_mean, rate_vol, months, dividend_yield=0.02, mode="PFU", tmi=0.30, reinvest_dividends=True):
    values = []
    total_invested = initial
    value = initial
    total_tax = 0
    cash_dividends = []
    rng = np.random.default_rng()
    
    # Choix fiscalité
    tax_rate = 0.30 if mode=="PFU" else tmi + 0.172
    
    for i in range(months):
        rate = rng.normal(rate_mean, rate_vol)
        value = value * (1 + rate) + monthly
        total_invested += monthly
        
        # Dividendes annuels
        if (i + 1) % 12 == 0 and dividend_yield > 0:
            dividends = value * dividend_yield
            tax = dividends * tax_rate
            net_dividends = dividends - tax
            if reinvest_dividends:
                value += net_dividends
                cash_dividends.append(0)
            else:
                cash_dividends.append(net_dividends)
            total_tax += tax
        
        values.append(value)
    
    net_value = value
    net_gain = net_value - total_invested
    return net_value, total_invested, net_gain, total_tax, values, cash_dividends

# --- Fonction interactive avancée ---
def simulate_advanced(capital_initial=2000, versement_mensuel=200, annual_return=0.08, volatility=0.02,
                      years=10, dividend_yield=0.03, reinvest_dividends=True, fiscalite_cto="PFU", tmi=0.30):
    months = years * 12
    monthly_mean = (1 + annual_return) ** (1/12) - 1
    monthly_vol = volatility / np.sqrt(12)  # conversion annual -> monthly
    
    # PEA
    pea_net, pea_invested, pea_gain_net, pea_tax, pea_values, pea_values_net, pea_cash_divs = simulate_pea(
        capital_initial, versement_mensuel, monthly_mean, monthly_vol, months,
        dividend_yield, reinvest_dividends
    )
    
    # CTO
    cto_net, cto_invested, cto_gain_net, cto_tax, cto_values, cto_cash_divs = simulate_cto(
        capital_initial, versement_mensuel, monthly_mean, monthly_vol, months,
        dividend_yield, fiscalite_cto, tmi, reinvest_dividends
    )
    
    # Tableau comparatif
    df = pd.DataFrame({
        "Compte": ["PEA", f"CTO ({fiscalite_cto})"],
        "Capital investi (€)": [pea_invested, cto_invested],
        "Impôt payé (€)": [pea_tax, cto_tax],
        "Gain net (€)": [pea_gain_net, cto_gain_net],
        "Valeur nette finale (€)": [pea_net, cto_net],
        "Dividendes encaissés (€)": [sum(pea_cash_divs), sum(cto_cash_divs)]
    })
    display(df)
    
    # Graphiques
    plt.figure(figsize=(12, 6))
    plt.plot(pea_values_net, label=f"PEA net", color="green")
    plt.plot(cto_values, label=f"CTO ({fiscalite_cto})", color="red")
    plt.plot(np.cumsum(pea_cash_divs)+capital_initial, '--', color="lightgreen", label="Dividendes PEA encaissés")
    plt.plot(np.cumsum(cto_cash_divs)+capital_initial, '--', color="orange", label="Dividendes CTO encaissés")
    plt.title(f"Simulation portefeuille PEA vs CTO sur {years} ans")
    plt.xlabel("Mois")
    plt.ylabel("Valeur du portefeuille (€)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# --- Interface interactive ---
interact(
    simulate_advanced,
    capital_initial=widgets.IntSlider(value=10000, min=0, max=25000, step=500, description="Capital initial (€)"),
    versement_mensuel=widgets.IntSlider(value=100, min=0, max=2000, step=50, description="Versement mensuel (€)"),
    annual_return=widgets.FloatSlider(value=0.08, min=0.01, max=0.20, step=0.01, description="Rendement moyen"),
    volatility=widgets.FloatSlider(value=0.02, min=0.0, max=0.10, step=0.005, description="Volatilité annuelle"),
    years=widgets.IntSlider(value=5, min=1, max=30, step=1, description="Durée (années)"),
    dividend_yield=widgets.FloatSlider(value=0.03, min=0.0, max=0.10, step=0.005, description="Dividendes (%)"),
    reinvest_dividends=widgets.Checkbox(value=True, description="Réinvestir dividendes"),
    fiscalite_cto=widgets.RadioButtons(options=["PFU", "Barème"], value="PFU", description="Fiscalité CTO"),
    tmi=widgets.Dropdown(options=[0.0, 0.11, 0.30, 0.41, 0.45], value=0.30, description="TMI (si barème)")
)


interactive(children=(IntSlider(value=10000, description='Capital initial (€)', max=25000, step=500), IntSlide…

<function __main__.simulate_advanced(capital_initial=2000, versement_mensuel=200, annual_return=0.08, volatility=0.02, years=10, dividend_yield=0.03, reinvest_dividends=True, fiscalite_cto='PFU', tmi=0.3)>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

# --- Monte Carlo PEA ---
def simulate_pea_mc(initial, monthly, rate_mean, rate_vol, months, dividend_yield=0.02, reinvest_dividends=True, tax_rate=0.172, n_sim=100):
    all_values = np.zeros((n_sim, months))
    rng = np.random.default_rng()
    
    for sim in range(n_sim):
        value = initial
        for i in range(months):
            rate = rng.normal(rate_mean, rate_vol)
            value = value * (1 + rate) + monthly
            if (i+1)%12 == 0 and dividend_yield>0:
                dividends = value*dividend_yield
                if reinvest_dividends:
                    value += dividends
        gain = value - (initial + monthly*months)
        net_value = value - gain*tax_rate if gain>0 else value
        all_values[sim,:] = value - ((value - (initial + monthly*months))*tax_rate)  # courbe nette prorata
    return all_values

# --- Monte Carlo CTO ---
def simulate_cto_mc(initial, monthly, rate_mean, rate_vol, months, dividend_yield=0.02, mode="PFU", tmi=0.30, reinvest_dividends=True, n_sim=100):
    all_values = np.zeros((n_sim, months))
    rng = np.random.default_rng()
    tax_rate = 0.30 if mode=="PFU" else tmi + 0.172
    
    for sim in range(n_sim):
        value = initial
        for i in range(months):
            rate = rng.normal(rate_mean, rate_vol)
            value = value*(1+rate) + monthly
            if (i+1)%12==0 and dividend_yield>0:
                dividends = value*dividend_yield
                net_div = dividends*(1-tax_rate)
                if reinvest_dividends:
                    value += net_div
        all_values[sim,:] = value
    return all_values

# --- Fonction interactive Monte Carlo ---
def simulate_mc(capital_initial=2000, versement_mensuel=200, annual_return=0.08, volatility=0.02,
                years=10, dividend_yield=0.03, reinvest_dividends=True, fiscalite_cto="PFU", tmi=0.30, n_sim=200):
    
    months = years*12
    monthly_mean = (1 + annual_return)**(1/12) - 1
    monthly_vol = volatility / np.sqrt(12)
    
    # Simulations Monte Carlo
    pea_mc = simulate_pea_mc(capital_initial, versement_mensuel, monthly_mean, monthly_vol, months,
                             dividend_yield, reinvest_dividends, 0.172, n_sim)
    cto_mc = simulate_cto_mc(capital_initial, versement_mensuel, monthly_mean, monthly_vol, months,
                             dividend_yield, fiscalite_cto, tmi, reinvest_dividends, n_sim)
    
    # Calcul des percentiles
    pea_mean = np.mean(pea_mc, axis=0)
    pea_p5 = np.percentile(pea_mc, 5, axis=0)
    pea_p95 = np.percentile(pea_mc, 95, axis=0)
    
    cto_mean = np.mean(cto_mc, axis=0)
    cto_p5 = np.percentile(cto_mc, 5, axis=0)
    cto_p95 = np.percentile(cto_mc, 95, axis=0)
    
    # Graphique Monte Carlo
    plt.figure(figsize=(12,6))
    
    plt.fill_between(range(months), pea_p5, pea_p95, color="green", alpha=0.2, label="PEA 5-95 percentile")
    plt.plot(pea_mean, color="green", label="PEA moyenne")
    
    plt.fill_between(range(months), cto_p5, cto_p95, color="red", alpha=0.2, label=f"CTO ({fiscalite_cto}) 5-95 percentile")
    plt.plot(cto_mean, color="red", label=f"CTO ({fiscalite_cto}) moyenne")
    
    plt.title(f"Monte Carlo Simulation PEA vs CTO ({years} ans, {n_sim} trajectoires)")
    plt.xlabel("Mois")
    plt.ylabel("Valeur du portefeuille (€)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# --- Interface interactive Monte Carlo ---
interact(
    simulate_mc,
    capital_initial=widgets.IntSlider(value=2000, min=0, max=20000, step=500, description="Capital initial (€)"),
    versement_mensuel=widgets.IntSlider(value=200, min=0, max=2000, step=50, description="Versement mensuel (€)"),
    annual_return=widgets.FloatSlider(value=0.08, min=0.01, max=0.20, step=0.01, description="Rendement moyen"),
    volatility=widgets.FloatSlider(value=0.02, min=0.0, max=0.10, step=0.005, description="Volatilité annuelle"),
    years=widgets.IntSlider(value=10, min=1, max=30, step=1, description="Durée (années)"),
    dividend_yield=widgets.FloatSlider(value=0.03, min=0.0, max=0.10, step=0.005, description="Dividendes (%)"),
    reinvest_dividends=widgets.Checkbox(value=True, description="Réinvestir dividendes"),
    fiscalite_cto=widgets.RadioButtons(options=["PFU", "Barème"], value="PFU", description="Fiscalité CTO"),
    tmi=widgets.Dropdown(options=[0.0, 0.11, 0.30, 0.41, 0.45], value=0.30, description="TMI (si barème)"),
    n_sim=widgets.IntSlider(value=200, min=50, max=1000, step=50, description="N simulations")
)


interactive(children=(IntSlider(value=2000, description='Capital initial (€)', max=20000, step=500), IntSlider…

<function __main__.simulate_mc(capital_initial=2000, versement_mensuel=200, annual_return=0.08, volatility=0.02, years=10, dividend_yield=0.03, reinvest_dividends=True, fiscalite_cto='PFU', tmi=0.3, n_sim=200)>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

# --- Monte Carlo PEA ---
def simulate_pea_mc(initial, monthly, rate_mean, rate_vol, months, dividend_yield=0.02,
                     reinvest_dividends=True, tax_rate=0.172, n_sim=100):
    all_values = np.zeros((n_sim, months))
    all_dividends = np.zeros((n_sim, months))
    rng = np.random.default_rng()
    
    for sim in range(n_sim):
        value = initial
        dividends_encaisse = 0
        for i in range(months):
            rate = rng.normal(rate_mean, rate_vol)
            value = value*(1+rate) + monthly
            
            if (i+1)%12==0 and dividend_yield>0:
                dividends = value*dividend_yield
                if reinvest_dividends:
                    value += dividends
                else:
                    dividends_encaisse += dividends
                    all_dividends[sim,i] = dividends
            
            # Courbe nette mois par mois
            gain = value - (initial + monthly*months)
            net_value = value - gain*tax_rate if gain>0 else value
            all_values[sim,i] = net_value
    
    return all_values, all_dividends

# --- Monte Carlo CTO ---
def simulate_cto_mc(initial, monthly, rate_mean, rate_vol, months, dividend_yield=0.02,
                    mode="PFU", tmi=0.30, reinvest_dividends=True, n_sim=100):
    all_values = np.zeros((n_sim, months))
    all_dividends = np.zeros((n_sim, months))
    rng = np.random.default_rng()
    tax_rate = 0.30 if mode=="PFU" else tmi+0.172
    
    for sim in range(n_sim):
        value = initial
        dividends_encaisse = 0
        for i in range(months):
            rate = rng.normal(rate_mean, rate_vol)
            value = value*(1+rate) + monthly
            
            if (i+1)%12==0 and dividend_yield>0:
                dividends = value*dividend_yield
                net_div = dividends*(1-tax_rate)
                if reinvest_dividends:
                    value += net_div
                else:
                    dividends_encaisse += net_div
                    all_dividends[sim,i] = net_div
            all_values[sim,i] = value
    return all_values, all_dividends

# --- Fonction interactive Monte Carlo complète ---
def simulate_mc_full(capital_initial=2000, versement_mensuel=200, annual_return=0.08, volatility=0.02,
                     years=10, dividend_yield=0.03, reinvest_dividends=True, fiscalite_cto="PFU",
                     tmi=0.30, n_sim=200):
    
    months = years*12
    monthly_mean = (1 + annual_return)**(1/12) - 1
    monthly_vol = volatility / np.sqrt(12)
    
    # Simulations PEA et CTO
    pea_mc, pea_divs = simulate_pea_mc(capital_initial, versement_mensuel, monthly_mean, monthly_vol,
                                       months, dividend_yield, reinvest_dividends, 0.172, n_sim)
    cto_mc, cto_divs = simulate_cto_mc(capital_initial, versement_mensuel, monthly_mean, monthly_vol,
                                       months, dividend_yield, fiscalite_cto, tmi, reinvest_dividends, n_sim)
    
    # Calcul moyennes et percentiles
    pea_mean = np.mean(pea_mc, axis=0)
    pea_p5 = np.percentile(pea_mc, 5, axis=0)
    pea_p95 = np.percentile(pea_mc, 95, axis=0)
    cto_mean = np.mean(cto_mc, axis=0)
    cto_p5 = np.percentile(cto_mc, 5, axis=0)
    cto_p95 = np.percentile(cto_mc, 95, axis=0)
    
    # Tableau récapitulatif
    df = pd.DataFrame({
        "Compte": ["PEA", f"CTO ({fiscalite_cto})"],
        "Capital investi (€)": [capital_initial + versement_mensuel*months]*2,
        "Gain net moyen (€)": [pea_mean[-1]-capital_initial-versement_mensuel*months,
                               cto_mean[-1]-capital_initial-versement_mensuel*months],
        "Dividendes encaissés (€)": [pea_divs.sum()/n_sim, cto_divs.sum()/n_sim],
        "Valeur nette finale moyenne (€)": [pea_mean[-1], cto_mean[-1]]
    })
    display(df)
    
    # Graphique Monte Carlo avec zone de confiance
    plt.figure(figsize=(12,6))
    
    # PEA
    plt.fill_between(range(months), pea_p5, pea_p95, color="green", alpha=0.2, label="PEA 5-95 percentile")
    plt.plot(pea_mean, color="green", label="PEA moyenne")
    # CTO
    plt.fill_between(range(months), cto_p5, cto_p95, color="red", alpha=0.2, label=f"CTO ({fiscalite_cto}) 5-95 percentile")
    plt.plot(cto_mean, color="red", label=f"CTO ({fiscalite_cto}) moyenne")
    
    plt.title(f"Monte Carlo PEA vs CTO ({years} ans, {n_sim} trajectoires)")
    plt.xlabel("Mois")
    plt.ylabel("Valeur du portefeuille (€)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# --- Interface interactive complète ---
interact(
    simulate_mc_full,
    capital_initial=widgets.IntSlider(value=2000, min=0, max=20000, step=500, description="Capital initial (€)"),
    versement_mensuel=widgets.IntSlider(value=200, min=0, max=2000, step=50, description="Versement mensuel (€)"),
    annual_return=widgets.FloatSlider(value=0.08, min=0.01, max=0.20, step=0.01, description="Rendement moyen"),
    volatility=widgets.FloatSlider(value=0.02, min=0.0, max=0.10, step=0.005, description="Volatilité annuelle"),
    years=widgets.IntSlider(value=10, min=1, max=30, step=1, description="Durée (années)"),
    dividend_yield=widgets.FloatSlider(value=0.03, min=0.0, max=0.10, step=0.005, description="Dividendes (%)"),
    reinvest_dividends=widgets.Checkbox(value=True, description="Réinvestir dividendes"),
    fiscalite_cto=widgets.RadioButtons(options=["PFU", "Barème"], value="PFU", description="Fiscalité CTO"),
    tmi=widgets.Dropdown(options=[0.0, 0.11, 0.30, 0.41, 0.45], value=0.30, description="TMI (si barème)"),
    n_sim=widgets.IntSlider(value=200, min=50, max=1000, step=50, description="N simulations")
)



interactive(children=(IntSlider(value=2000, description='Capital initial (€)', max=20000, step=500), IntSlider…

<function __main__.simulate_mc_full(capital_initial=2000, versement_mensuel=200, annual_return=0.08, volatility=0.02, years=10, dividend_yield=0.03, reinvest_dividends=True, fiscalite_cto='PFU', tmi=0.3, n_sim=200)>

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

# --- Simulation Monte Carlo pour PEA ---
def simulate_pea_mc(initial, monthly, rate_mean, rate_vol, months, dividend_yield=0.02,
                     reinvest_dividends=True, tax_rate=0.172, n_sim=100):
    all_values = np.zeros((n_sim, months))
    all_dividends = np.zeros((n_sim, months))
    rng = np.random.default_rng()
    
    for sim in range(n_sim):
        value = initial
        dividends_encaisse = 0
        for i in range(months):
            rate = rng.normal(rate_mean, rate_vol)
            value = value*(1+rate) + monthly
            if (i+1)%12==0 and dividend_yield>0:
                dividends = value*dividend_yield
                if reinvest_dividends:
                    value += dividends
                else:
                    dividends_encaisse += dividends
                    all_dividends[sim,i] = dividends
            gain = value - (initial + monthly*months)
            net_value = value - gain*tax_rate if gain>0 else value
            all_values[sim,i] = net_value
    return all_values, all_dividends

# --- Simulation Monte Carlo pour CTO ---
def simulate_cto_mc(initial, monthly, rate_mean, rate_vol, months, dividend_yield=0.02,
                    mode="PFU", tmi=0.30, reinvest_dividends=True, n_sim=100):
    all_values = np.zeros((n_sim, months))
    all_dividends = np.zeros((n_sim, months))
    rng = np.random.default_rng()
    tax_rate = 0.30 if mode=="PFU" else tmi+0.172
    
    for sim in range(n_sim):
        value = initial
        dividends_encaisse = 0
        for i in range(months):
            rate = rng.normal(rate_mean, rate_vol)
            value = value*(1+rate) + monthly
            if (i+1)%12==0 and dividend_yield>0:
                dividends = value*dividend_yield
                net_div = dividends*(1-tax_rate)
                if reinvest_dividends:
                    value += net_div
                else:
                    dividends_encaisse += net_div
                    all_dividends[sim,i] = net_div
            all_values[sim,i] = value
    return all_values, all_dividends

# --- Fonction ultime interactive ---
def simulate_mc_ultimate(capital_initial=2000, versement_mensuel=200, annual_return=0.08, volatility=0.02,
                         years=10, dividend_yield=0.03, reinvest_dividends=True, fiscalite_cto="PFU",
                         tmi=0.30, n_sim=200, seuil=50000):
    
    months = years*12
    monthly_mean = (1 + annual_return)**(1/12) - 1
    monthly_vol = volatility / np.sqrt(12)
    
    # Monte Carlo
    pea_mc, pea_divs = simulate_pea_mc(capital_initial, versement_mensuel, monthly_mean, monthly_vol,
                                       months, dividend_yield, reinvest_dividends, 0.172, n_sim)
    cto_mc, cto_divs = simulate_cto_mc(capital_initial, versement_mensuel, monthly_mean, monthly_vol,
                                       months, dividend_yield, fiscalite_cto, tmi, reinvest_dividends, n_sim)
    
    # Moyennes et percentiles
    pea_mean, pea_p5, pea_p95 = np.mean(pea_mc, axis=0), np.percentile(pea_mc,5,axis=0), np.percentile(pea_mc,95,axis=0)
    cto_mean, cto_p5, cto_p95 = np.mean(cto_mc, axis=0), np.percentile(cto_mc,5,axis=0), np.percentile(cto_mc,95,axis=0)
    
    # Tableau récapitulatif
    df = pd.DataFrame({
        "Compte": ["PEA", f"CTO ({fiscalite_cto})"],
        "Capital investi (€)": [capital_initial + versement_mensuel*months]*2,
        "Gain net moyen (€)": [pea_mean[-1]-capital_initial-versement_mensuel*months,
                               cto_mean[-1]-capital_initial-versement_mensuel*months],
        "Dividendes encaissés (€)": [pea_divs.sum()/n_sim, cto_divs.sum()/n_sim],
        "Valeur nette finale moyenne (€)": [pea_mean[-1], cto_mean[-1]],
        "Probabilité > seuil (€)": [(pea_mc[:,-1]>seuil).mean()*100, (cto_mc[:,-1]>seuil).mean()*100]
    })
    display(df)
    
    # Graphiques
    plt.figure(figsize=(14,6))
    
    plt.fill_between(range(months), pea_p5, pea_p95, color="green", alpha=0.2, label="PEA 5-95 percentile")
    plt.plot(pea_mean, color="green", label="PEA moyenne")
    
    plt.fill_between(range(months), cto_p5, cto_p95, color="red", alpha=0.2, label=f"CTO ({fiscalite_cto}) 5-95 percentile")
    plt.plot(cto_mean, color="red", label=f"CTO ({fiscalite_cto}) moyenne")
    
    # Histogramme des gains finaux
    plt.figure(figsize=(12,5))
    plt.hist(pea_mc[:,-1], bins=30, alpha=0.5, label="PEA final")
    plt.hist(cto_mc[:,-1], bins=30, alpha=0.5, label=f"CTO ({fiscalite_cto}) final")
    plt.axvline(seuil, color='black', linestyle='--', label=f"Seuil {seuil}€")
    plt.title("Distribution des valeurs finales")
    plt.xlabel("Valeur finale (€)")
    plt.ylabel("Nombre de trajectoires")
    plt.legend()
    plt.grid(True)
    plt.show()

# --- Interface ultime ---
interact(
    simulate_mc_ultimate,
    capital_initial=widgets.IntSlider(value=2000, min=0, max=20000, step=500, description="Capital initial (€)"),
    versement_mensuel=widgets.IntSlider(value=200, min=0, max=2000, step=50, description="Versement mensuel (€)"),
    annual_return=widgets.FloatSlider(value=0.08, min=0.01, max=0.20, step=0.01, description="Rendement moyen"),
    volatility=widgets.FloatSlider(value=0.02, min=0.0, max=0.10, step=0.005, description="Volatilité annuelle"),
    years=widgets.IntSlider(value=10, min=1, max=30, step=1, description="Durée (années)"),
    dividend_yield=widgets.FloatSlider(value=0.03, min=0.0, max=0.10, step=0.005, description="Dividendes (%)"),
    reinvest_dividends=widgets.Checkbox(value=True, description="Réinvestir dividendes"),
    fiscalite_cto=widgets.RadioButtons(options=["PFU", "Barème"], value="PFU", description="Fiscalité CTO"),
    tmi=widgets.Dropdown(options=[0.0, 0.11, 0.30, 0.41, 0.45], value=0.30, description="TMI (si barème)"),
    n_sim=widgets.IntSlider(value=200, min=50, max=1000, step=50, description="N simulations"),
    seuil=widgets.IntSlider(value=50000, min=10000, max=500000, step=5000, description="Seuil (€)")
)


interactive(children=(IntSlider(value=2000, description='Capital initial (€)', max=20000, step=500), IntSlider…

<function __main__.simulate_mc_ultimate(capital_initial=2000, versement_mensuel=200, annual_return=0.08, volatility=0.02, years=10, dividend_yield=0.03, reinvest_dividends=True, fiscalite_cto='PFU', tmi=0.3, n_sim=200, seuil=50000)>